In [138]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [139]:
from pearce.emulator import OriginalRecipe, ExtraCrispy
from pearce.mocks import cat_dict
import numpy as np
from os import path

In [140]:
import tensorflow as tf

In [141]:
import matplotlib
#matplotlib.use('Agg')
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [142]:
training_file = '/home/sean/PearceRedMagicXiCosmo.hdf5'
test_file = '/home/sean/PearceRedMagicXiCosmoTest.hdf5'

In [143]:
em_method = 'nn'
split_method = 'random'

In [144]:
a = 1.0
z = 1.0/a - 1.0

In [145]:
fixed_params = {'z':z}#, 'r':17.0389993 }

In [146]:
emu = OriginalRecipe(training_file, method = em_method, fixed_params=fixed_params,
                    hyperparams = {'hidden_layer_sizes': (10),
                                 'activation': 'relu', 'verbose': True, 
                                    'tol': 1e-8, 'learning_rate_init':1e-4,\
                                   'max_iter':10, 'alpha':0, 'early_stopping':False, 'validation_fraction':0.3})

Iteration 1, loss = 0.32825790
Iteration 2, loss = 0.10845689
Iteration 3, loss = 0.08724234
Iteration 4, loss = 0.08313204
Iteration 5, loss = 0.08168116
Iteration 6, loss = 0.08096126
Iteration 7, loss = 0.08054406
Iteration 8, loss = 0.08027376
Iteration 9, loss = 0.08007939
Iteration 10, loss = 0.07993874


In [147]:
#idxs = np.random.choice(emu.x.shape[0], size = int(emu.x.shape[0]*1.0), replace=False)

#x_train, y_train,yerr_train = emu.x[idxs, :],emu.y[idxs],emu.yerr[idxs]
x_train, y_train,yerr_train = emu.x,emu.y,emu.yerr

y_train = y_train*(emu._y_std + 1e-5) + emu._y_mean
yerr_train = yerr_train*(emu._y_std+1e-5)

In [148]:
x_train = x_train[0:-1:emu.n_bins, :-1]

y_train = y_train.reshape([-1, emu.n_bins])
yerr_train = yerr_train.reshape([-1, emu.n_bins])

In [149]:
emu.get_param_names()[:7]

['ombh2', 'omch2', 'w0', 'ns', 'ln10As', 'H0', 'Neff']

In [150]:
unique_cosmos = np.unique(x_train[:, :7], axis =0)#*(emu._x_std[:7]+1e-5) + emu._x_mean[:7]

In [151]:
unique_cosmos.shape

(40, 7)

In [152]:
x_test, y_test, ycov_test, _ = emu.get_data(test_file, fixed_params, None, False)
x_test = (x_test - emu._x_mean)/(emu._x_std+1e-5)

#split_ycov = np.dsplit(ycov_test, ycov_test.shape[-1])
        #fullcov = block_diag(*[yc[:,:,0] for yc in split_ycov])
#yerr_test = np.sqrt(np.hstack(np.diag(syc[:,:,0]) for syc in split_ycov))

In [153]:
x_test = x_test[0:-1:emu.n_bins, :-1]

y_test = y_test.reshape([-1, emu.n_bins])
#yerr_test = yerr_test.reshape([-1, emu.n_bins])

In [154]:
def n_layer_fc(x, hidden_sizes, training=False, l = 1e-8):
    initializer = tf.variance_scaling_initializer(scale=2.0)
    regularizer = tf.contrib.layers.l2_regularizer(l)
    fc_output = tf.layers.dense(x, hidden_sizes[0], activation=tf.nn.relu,
                                 kernel_initializer = initializer, kernel_regularizer = regularizer)
                                 #kernel_regularizer = tf.nn.l2_loss)
    #fc2_output = tf.layers.dense(fc1_output, hidden_sizes[1], activation=tf.nn.relu,
    #                             kernel_initializer = initializer, kernel_regularizer = regularizer)
    for size in hidden_sizes[1:]:
        fc_output = tf.layers.dense(fc_output, size, activation=tf.nn.relu, kernel_initializer=initializer,
                                 kernel_regularizer = regularizer)
    pred = tf.layers.dense(fc_output, 1, kernel_initializer=initializer, 
                              kernel_regularizer = regularizer)[:,0]#,
    return pred

In [175]:
def n_layer_fc_do_bn(x, hidden_sizes, training=False, l = 1e2):
    initializer = tf.variance_scaling_initializer(scale=2.0)
    regularizer = tf.contrib.layers.l1_regularizer(l)
    nl_out = x

    for size in hidden_sizes:
        fc_output = tf.layers.dense(nl_out, size,
                                 kernel_initializer = initializer, kernel_regularizer = regularizer)
        bd_out = tf.layers.dropout(fc_output, 0.333, training = training)
        bn_out = tf.layers.batch_normalization(bd_out, axis = -1, training=training)
        nl_out = tf.nn.relu(bn_out)#tf.nn.leaky_relu(bn_out, alpha=0.01)
    
    pred = tf.layers.dense(nl_out, emu.n_bins, kernel_initializer=initializer, 
                              kernel_regularizer = regularizer)#[:,0]#,
    return pred

In [176]:
def optimizer_init_fn(learning_rate = 1e-7):
    return tf.train.AdamOptimizer(learning_rate)#, beta1=0.9, beta2=0.999, epsilon=1e-6)

In [177]:
from sklearn.metrics import r2_score, mean_squared_error

In [178]:
def check_accuracy(sess, val_data,batch_size, x, weights, preds, is_training=None):
    val_x, val_y = val_data
    perc_acc, scores = [],[]
    for idx in xrange(0, val_x.shape[0], batch_size):
        feed_dict = {x: val_x[idx:idx+batch_size],
                     is_training: 0}
        y_pred = sess.run(preds, feed_dict=feed_dict)
        #print y_pred.shape, val_y[idx:idx+batch_size].shape
        score = r2_score(val_y[idx:idx+batch_size], y_pred)
        scores.append(score)
        
        perc_acc = np.mean(emu._y_std*np.abs(val_y[idx:idx+batch_size]-y_pred)/np.abs(emu._y_std*val_y[idx:idx+batch_size] + emu._y_mean) )

        
    print 'Val score: %.6f, %.2f %% Loss'%(np.mean(np.array(scores)), 100*np.mean(np.array(perc_acc)))

In [179]:
device = '/device:GPU:0'
#device = '/cpu:0'
def train(model_init_fn, optimizer_init_fn,num_params, train_data, val_data, hidden_sizes,\
               num_epochs=1, batch_size = 200, print_every=10):
    tf.reset_default_graph()    
    with tf.device(device):
        # Construct the computational graph we will use to train the model. We
        # use the model_init_fn to construct the model, declare placeholders for
        # the data and labels
        x = tf.placeholder(tf.float32, [None,num_params], name = 'x')
        y = tf.placeholder(tf.float32, [None, emu.n_bins], name = 'y')
        weights = tf.placeholder(tf.float32, [None, emu.n_bins], name = 'weights')
        
        is_training = tf.placeholder(tf.bool, name='is_training')
        
        preds = model_init_fn(x, hidden_sizes, is_training)

        # Compute the loss like we did in Part II
        #loss = tf.reduce_mean(loss)
        
    with tf.device('/cpu:0'):
        loss = tf.losses.mean_squared_error(labels=y,\
                    predictions=preds, weights = weights)#weights?
        #loss = tf.losses.absolute_difference(labels=y, predictions=preds, weights = tf.abs(1.0/y))#weights?

    with tf.device(device):
        optimizer = optimizer_init_fn()
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            train_op = optimizer.minimize(loss)
        
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        #t = 0
        train_x, train_y, train_yerr = train_data
        rand_idxs = range(train_x.shape[0])
        for epoch in range(num_epochs):
            #print('Starting epoch %d' % epoch)
            np.random.shuffle(rand_idxs)
            losses = []
            for idx in xrange(0, train_x.shape[0], batch_size):
                _bs = train_x[rand_idxs[idx:idx+batch_size]].shape[0]
                feed_dict = {x: train_x[rand_idxs[idx:idx+batch_size]],\
                             y: train_y[rand_idxs[idx:idx+batch_size]] + np.random.randn(_bs, emu.n_bins)*train_yerr[rand_idxs[idx:idx+batch_size]],\
                             weights: 1/train_yerr[rand_idxs[idx:idx+batch_size]],\
                             is_training:1}
                loss_np, _, preds_np = sess.run([loss, train_op, preds], feed_dict=feed_dict)
                #preds_np = sess.run([preds], feed_dict=feed_dict)
                #print preds_np[0].shape
                losses.append(loss_np)
                
                
            if epoch % print_every == 0:
                loss_avg = np.mean(np.array(losses))
                print('Epoch %d, loss = %e' % (epoch, loss_avg))
                check_accuracy(sess, val_data, batch_size, x, weights, preds, is_training=is_training)
            #t += 1

In [180]:
train(n_layer_fc_do_bn, optimizer_init_fn, x_train.shape[1],\
           (x_train, y_train, yerr_train), (x_test, y_test),\
           [200, 500, 500,1000,500, 500, 200], num_epochs= int(1e4), batch_size = 100, \
           print_every = 10)

Epoch 0, loss = 1.396695e+03
Val score: -12.715815, 62.54 % Loss
Epoch 10, loss = 1.370375e+03
Val score: -12.722258, 62.38 % Loss
Epoch 20, loss = 1.348620e+03
Val score: -12.609831, 62.05 % Loss
Epoch 30, loss = 1.321280e+03
Val score: -12.493435, 61.80 % Loss
Epoch 40, loss = 1.296086e+03
Val score: -12.387121, 61.49 % Loss
Epoch 50, loss = 1.275644e+03
Val score: -12.246988, 61.21 % Loss
Epoch 60, loss = 1.252707e+03
Val score: -12.136255, 60.89 % Loss
Epoch 70, loss = 1.235530e+03
Val score: -11.936174, 60.30 % Loss
Epoch 80, loss = 1.213781e+03
Val score: -11.819302, 60.05 % Loss
Epoch 90, loss = 1.192854e+03
Val score: -11.664311, 59.67 % Loss
Epoch 100, loss = 1.173524e+03
Val score: -11.454376, 59.05 % Loss
Epoch 110, loss = 1.155185e+03
Val score: -11.354989, 58.74 % Loss
Epoch 120, loss = 1.137976e+03
Val score: -11.130107, 58.11 % Loss
Epoch 130, loss = 1.121424e+03
Val score: -10.926523, 57.49 % Loss
Epoch 140, loss = 1.101845e+03
Val score: -10.771099, 57.01 % Loss
Epoch 

Epoch 1240, loss = 2.083735e+02
Val score: -4.042878, 33.78 % Loss
Epoch 1250, loss = 2.060578e+02
Val score: -3.986312, 33.58 % Loss
Epoch 1260, loss = 2.018887e+02
Val score: -3.952912, 33.48 % Loss
Epoch 1270, loss = 1.987358e+02
Val score: -3.902921, 33.23 % Loss
Epoch 1280, loss = 1.964800e+02
Val score: -3.942853, 33.43 % Loss
Epoch 1290, loss = 1.935793e+02
Val score: -3.902930, 33.33 % Loss
Epoch 1300, loss = 1.914431e+02
Val score: -3.858679, 33.19 % Loss
Epoch 1310, loss = 1.883045e+02
Val score: -3.824035, 33.06 % Loss
Epoch 1320, loss = 1.855563e+02
Val score: -3.846383, 33.16 % Loss
Epoch 1330, loss = 1.829454e+02
Val score: -3.792781, 33.00 % Loss
Epoch 1340, loss = 1.814371e+02
Val score: -3.769845, 32.88 % Loss
Epoch 1350, loss = 1.771391e+02
Val score: -3.729591, 32.73 % Loss
Epoch 1360, loss = 1.753880e+02
Val score: -3.752314, 32.73 % Loss
Epoch 1370, loss = 1.743495e+02
Val score: -3.723012, 32.74 % Loss
Epoch 1380, loss = 1.710054e+02
Val score: -3.633008, 32.29 % 

Epoch 2470, loss = 9.965119e+01
Val score: -2.171038, 27.25 % Loss
Epoch 2480, loss = 9.917828e+01
Val score: -2.179177, 27.24 % Loss
Epoch 2490, loss = 9.960882e+01
Val score: -2.209034, 27.45 % Loss
Epoch 2500, loss = 9.909964e+01
Val score: -2.165418, 27.25 % Loss
Epoch 2510, loss = 9.903596e+01
Val score: -2.162885, 27.26 % Loss
Epoch 2520, loss = 9.917923e+01
Val score: -2.160438, 27.09 % Loss
Epoch 2530, loss = 9.898936e+01
Val score: -2.166319, 27.27 % Loss
Epoch 2540, loss = 9.836063e+01
Val score: -2.130549, 27.10 % Loss
Epoch 2550, loss = 9.839883e+01
Val score: -2.110451, 26.93 % Loss
Epoch 2560, loss = 9.794351e+01
Val score: -2.088506, 26.91 % Loss
Epoch 2570, loss = 9.772655e+01
Val score: -2.128097, 27.06 % Loss
Epoch 2580, loss = 9.796035e+01
Val score: -2.100055, 26.95 % Loss
Epoch 2590, loss = 9.751897e+01
Val score: -2.070937, 26.84 % Loss
Epoch 2600, loss = 9.776514e+01
Val score: -2.082129, 26.83 % Loss
Epoch 2610, loss = 9.743335e+01
Val score: -2.100771, 26.94 % 

Epoch 3700, loss = 8.241844e+01
Val score: -1.428732, 23.87 % Loss
Epoch 3710, loss = 8.306236e+01
Val score: -1.450036, 23.94 % Loss
Epoch 3720, loss = 8.252190e+01
Val score: -1.432713, 23.86 % Loss
Epoch 3730, loss = 8.190011e+01
Val score: -1.410175, 23.73 % Loss
Epoch 3740, loss = 8.237660e+01
Val score: -1.399827, 23.70 % Loss
Epoch 3750, loss = 8.206761e+01
Val score: -1.429891, 23.81 % Loss
Epoch 3760, loss = 8.231115e+01
Val score: -1.407094, 23.76 % Loss
Epoch 3770, loss = 8.187096e+01
Val score: -1.399386, 23.70 % Loss
Epoch 3780, loss = 8.226395e+01
Val score: -1.398778, 23.65 % Loss
Epoch 3790, loss = 8.183837e+01
Val score: -1.363282, 23.54 % Loss
Epoch 3800, loss = 8.153862e+01
Val score: -1.392632, 23.64 % Loss
Epoch 3810, loss = 8.189048e+01
Val score: -1.396420, 23.69 % Loss
Epoch 3820, loss = 8.146529e+01
Val score: -1.393151, 23.65 % Loss
Epoch 3830, loss = 8.092895e+01
Val score: -1.376071, 23.52 % Loss
Epoch 3840, loss = 8.110699e+01
Val score: -1.357316, 23.44 % 

Epoch 4930, loss = 7.293774e+01
Val score: -0.996176, 21.41 % Loss
Epoch 4940, loss = 7.268549e+01
Val score: -0.992165, 21.44 % Loss
Epoch 4950, loss = 7.250778e+01
Val score: -0.986528, 21.40 % Loss
Epoch 4960, loss = 7.238806e+01
Val score: -0.985690, 21.35 % Loss
Epoch 4970, loss = 7.220260e+01
Val score: -0.982698, 21.35 % Loss
Epoch 4980, loss = 7.234345e+01
Val score: -0.958193, 21.31 % Loss
Epoch 4990, loss = 7.247191e+01
Val score: -0.979360, 21.34 % Loss
Epoch 5000, loss = 7.224237e+01
Val score: -0.972040, 21.31 % Loss
Epoch 5010, loss = 7.252645e+01
Val score: -0.964318, 21.24 % Loss
Epoch 5020, loss = 7.223064e+01
Val score: -0.974421, 21.29 % Loss
Epoch 5030, loss = 7.219926e+01
Val score: -0.961472, 21.25 % Loss
Epoch 5040, loss = 7.209634e+01
Val score: -0.969698, 21.28 % Loss
Epoch 5050, loss = 7.197521e+01
Val score: -0.956387, 21.27 % Loss
Epoch 5060, loss = 7.164151e+01
Val score: -0.939389, 21.15 % Loss
Epoch 5070, loss = 7.158199e+01
Val score: -0.944525, 21.15 % 

Epoch 6160, loss = 6.584721e+01
Val score: -0.760224, 20.09 % Loss
Epoch 6170, loss = 6.574802e+01
Val score: -0.767871, 20.13 % Loss
Epoch 6180, loss = 6.612467e+01
Val score: -0.757858, 20.02 % Loss
Epoch 6190, loss = 6.580331e+01
Val score: -0.754238, 20.03 % Loss
Epoch 6200, loss = 6.591731e+01
Val score: -0.758423, 20.06 % Loss
Epoch 6210, loss = 6.582111e+01
Val score: -0.754976, 20.00 % Loss
Epoch 6220, loss = 6.587987e+01
Val score: -0.756409, 20.01 % Loss
Epoch 6230, loss = 6.583056e+01
Val score: -0.747381, 19.97 % Loss
Epoch 6240, loss = 6.574294e+01
Val score: -0.753395, 20.00 % Loss
Epoch 6250, loss = 6.558680e+01
Val score: -0.746568, 20.00 % Loss
Epoch 6260, loss = 6.595879e+01
Val score: -0.760088, 20.04 % Loss
Epoch 6270, loss = 6.582130e+01
Val score: -0.752487, 19.98 % Loss
Epoch 6280, loss = 6.552458e+01
Val score: -0.745759, 20.01 % Loss
Epoch 6290, loss = 6.555289e+01
Val score: -0.745410, 19.97 % Loss
Epoch 6300, loss = 6.535790e+01
Val score: -0.752313, 20.02 % 

Epoch 7390, loss = 6.188707e+01
Val score: -0.677686, 19.54 % Loss
Epoch 7400, loss = 6.080898e+01
Val score: -0.684586, 19.60 % Loss
Epoch 7410, loss = 6.125956e+01
Val score: -0.668446, 19.53 % Loss
Epoch 7420, loss = 6.164423e+01
Val score: -0.683546, 19.58 % Loss
Epoch 7430, loss = 6.132212e+01
Val score: -0.678453, 19.54 % Loss
Epoch 7440, loss = 6.177175e+01
Val score: -0.670163, 19.51 % Loss
Epoch 7450, loss = 6.125412e+01
Val score: -0.680928, 19.57 % Loss
Epoch 7460, loss = 6.130932e+01
Val score: -0.675801, 19.54 % Loss
Epoch 7470, loss = 6.146381e+01
Val score: -0.671973, 19.52 % Loss
Epoch 7480, loss = 6.098122e+01
Val score: -0.676892, 19.55 % Loss
Epoch 7490, loss = 6.160057e+01
Val score: -0.673029, 19.51 % Loss
Epoch 7500, loss = 6.118596e+01
Val score: -0.694134, 19.60 % Loss
Epoch 7510, loss = 6.176707e+01
Val score: -0.672821, 19.54 % Loss
Epoch 7520, loss = 6.097066e+01
Val score: -0.678141, 19.54 % Loss
Epoch 7530, loss = 6.137647e+01
Val score: -0.674203, 19.54 % 

Epoch 8620, loss = 5.880954e+01
Val score: -0.674538, 19.43 % Loss
Epoch 8630, loss = 5.836072e+01
Val score: -0.683201, 19.47 % Loss
Epoch 8640, loss = 5.892368e+01
Val score: -0.671686, 19.43 % Loss
Epoch 8650, loss = 5.838038e+01
Val score: -0.676864, 19.44 % Loss
Epoch 8660, loss = 5.835272e+01
Val score: -0.660669, 19.37 % Loss
Epoch 8670, loss = 5.840124e+01
Val score: -0.673719, 19.41 % Loss
Epoch 8680, loss = 5.823202e+01
Val score: -0.663591, 19.38 % Loss
Epoch 8690, loss = 5.848689e+01
Val score: -0.677642, 19.43 % Loss
Epoch 8700, loss = 5.847242e+01
Val score: -0.673946, 19.42 % Loss
Epoch 8710, loss = 5.823428e+01
Val score: -0.670725, 19.42 % Loss
Epoch 8720, loss = 5.836524e+01
Val score: -0.673792, 19.44 % Loss
Epoch 8730, loss = 5.847680e+01
Val score: -0.668225, 19.40 % Loss
Epoch 8740, loss = 5.851144e+01
Val score: -0.663054, 19.38 % Loss
Epoch 8750, loss = 5.828814e+01
Val score: -0.672811, 19.42 % Loss
Epoch 8760, loss = 5.820416e+01
Val score: -0.672218, 19.42 % 

Epoch 9850, loss = 5.665324e+01
Val score: -0.688519, 19.42 % Loss
Epoch 9860, loss = 5.623232e+01
Val score: -0.689466, 19.43 % Loss
Epoch 9870, loss = 5.677708e+01
Val score: -0.687880, 19.42 % Loss
Epoch 9880, loss = 5.615081e+01
Val score: -0.679071, 19.38 % Loss
Epoch 9890, loss = 5.659758e+01
Val score: -0.699400, 19.46 % Loss
Epoch 9900, loss = 5.653627e+01
Val score: -0.695067, 19.46 % Loss
Epoch 9910, loss = 5.639317e+01
Val score: -0.685550, 19.38 % Loss
Epoch 9920, loss = 5.627105e+01
Val score: -0.697463, 19.45 % Loss
Epoch 9930, loss = 5.626694e+01
Val score: -0.680098, 19.38 % Loss
Epoch 9940, loss = 5.596341e+01
Val score: -0.691442, 19.44 % Loss
Epoch 9950, loss = 5.630378e+01
Val score: -0.684863, 19.40 % Loss
Epoch 9960, loss = 5.640816e+01
Val score: -0.692069, 19.45 % Loss
Epoch 9970, loss = 5.626254e+01
Val score: -0.687647, 19.40 % Loss
Epoch 9980, loss = 5.659062e+01
Val score: -0.689532, 19.42 % Loss
Epoch 9990, loss = 5.652359e+01
Val score: -0.693296, 19.45 % 

In [ ]:
np.abs(emu.goodness_of_fit(training_file, statistic = 'log_frac')).mean()

In [ ]:
np.abs(emu.goodness_of_fit(training_file, statistic = 'frac')).mean()

In [ ]:
fit_idxs = np.argsort(gof.mean(axis = 1))

In [ ]:
emu.goodness_of_fit(training_file).mean()#, statistic = 'log_frac')).mean()

In [ ]:
model = emu._emulator

In [ ]:
ypred = model.predict(emu.x)

In [ ]:
plt.hist( np.log10( (emu._y_std+1e-5)*np.abs(ypred-emu.y)/np.abs((emu._y_std+1e-5)*emu.y+emu._y_mean) ))

In [ ]:
( (emu._y_std+1e-5)*np.abs(ypred-emu.y)/np.abs((emu._y_std+1e-5)*emu.y+emu._y_mean) ).mean()

In [ ]:
emu._y_mean, emu._y_std

In [ ]:
for idx in fit_idxs[:10]:
    print gof[idx].mean()
    print (ypred[idx*emu.n_bins:(idx+1)*emu.n_bins]-emu.y[idx*emu.n_bins:(idx+1)*emu.n_bins])/emu.y[idx*emu.n_bins:(idx+1)*emu.n_bins]
    plt.plot(emu.scale_bin_centers, ypred[idx*emu.n_bins:(idx+1)*emu.n_bins], label = 'Emu')
    plt.plot(emu.scale_bin_centers, emu.y[idx*emu.n_bins:(idx+1)*emu.n_bins], label = 'True')
    plt.legend(loc='best')
    plt.xscale('log')
    plt.show()

In [ ]:
print dict(zip(emu.get_param_names(), emu.x[8*emu.n_bins, :]*emu._x_std+emu._x_mean))

In [ ]:
emu.get_param_names()

In [ ]:
emu._ordered_params

In [ ]:
gof = emu.goodness_of_fit(training_file, statistic = 'frac')
print gof.mean()

In [ ]:
for row in gof:
    print row

In [ ]:
gof = emu.goodness_of_fit(training_file, statistic = 'frac')
print gof.mean()

In [ ]:
model = emu._emulator

In [ ]:
model.score(emu.x, emu.y)

In [ ]:
ypred = model.predict(emu.x)

np.mean(np.abs(ypred-emu.y)/emu.y)

In [ ]:
plt.plot(emu.scale_bin_centers, np.abs(gof.mean(axis = 0)) )
plt.plot(emu.scale_bin_centers, np.ones_like(emu.scale_bin_centers)*0.01)
plt.plot(emu.scale_bin_centers, np.ones_like(emu.scale_bin_centers)*0.05)
plt.plot(emu.scale_bin_centers, np.ones_like(emu.scale_bin_centers)*0.1)


plt.loglog();

In [ ]:
plt.plot(emu.scale_bin_centers, np.abs(gof.T),alpha = 0.1, color = 'b')
plt.plot(emu.scale_bin_centers, np.ones_like(emu.scale_bin_centers)*0.01, lw = 2, color = 'k')
plt.loglog();

In [ ]:
gof[:,i].shape

In [ ]:
for i in xrange(gof.shape[1]):
    plt.hist(np.log10(gof[:, i]), label = str(i), alpha = 0.2);
    
plt.legend(loc='best')
plt.show()